# Data exploration

Dit notebook zal worden gebruikt voor het vinden van interessante features.

In [ ]:
# imported libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# Kolommen die niet met n.v.t. of "?" werden aangegeven in de data dictionary.
# Zulke kolommen mochten wij negeren (volgens het interview)
cols_to_use= [
 '#stm_sap_meldnr',
 'stm_mon_nr',
 'stm_vl_post',
 'stm_sap_meld_ddt',
 'stm_sap_meldtekst_lang',
 'stm_sap_meldtekst',
 'stm_geo_mld',
 'stm_equipm_nr_mld',
 'stm_equipm_soort_mld',
 'stm_equipm_omschr_mld',
 'stm_km_van_mld',
 'stm_km_tot_mld',
 'stm_prioriteit',
 'stm_aanngeb_ddt',
 'stm_oh_pg_gst',
 'stm_geo_gst',
 'stm_equipm_nr_gst',
 'stm_equipm_soort_gst',
 'stm_equipm_omschr_gst',
 'stm_km_van_gst',
 'stm_km_tot_gst',
 'stm_oorz_groep',
 'stm_oorz_code',
 'stm_oorz_tkst',
 'stm_fh_ddt',
 'stm_fh_status',
 'stm_sap_storeind_ddt',
 'stm_tao_indicator',
 'stm_tao_indicator_vorige',
 'stm_tao_soort_mutatie',
 'stm_tao_telling_mutatie',
 'stm_tao_beinvloedbaar_indicator',
 'stm_sap_melddatum',
 'stm_sap_meldtijd',
 'stm_contractgeb_mld',
 'stm_techn_mld',
 'stm_contractgeb_gst',
 'stm_techn_gst',
 'stm_aanngeb_dd',
 'stm_aanngeb_tijd',
 'stm_aanntpl_dd',
 'stm_aanntpl_tijd',
 'stm_progfh_in_datum',
 'stm_progfh_in_tijd',
 'stm_progfh_in_invoer_dat',
 'stm_progfh_in_invoer_tijd',
 'stm_progfh_in_duur',
 'stm_progfh_gw_tijd',
 'stm_progfh_gw_lwd_datum',
 'stm_progfh_gw_lwd_tijd',
 'stm_progfh_gw_duur',
 'stm_progfh_gw_teller',
 'stm_fh_dd',
 'stm_fh_tijd',
 'stm_fh_duur',
 'stm_sap_storeinddatum',
 'stm_sap_storeindtijd',
 'stm_oorz_tekst_kort',
 'stm_pplg_van',
 'stm_pplg_naar',
 'stm_dstrglp_van',
 'stm_dstrglp_naar']

In [ ]:
df = pd.read_csv("data/sap_storing_data_hu_project.csv", index_col=0, usecols=cols_to_use, engine='pyarrow')

## Target variabele aanmaken

Target variabele anm_tot_fh, welke de duur vanaf dat de aannemer ter plaatse is, tot het daadwerkelijke functieherstel.

In [ ]:
# Convert columns to datetime type
df['stm_aanntpl_tijd'] = pd.to_datetime(df['stm_aanntpl_tijd'], format='%H:%M:%S', errors='coerce')
df['stm_aanntpl_dd'] = pd.to_datetime(df['stm_aanntpl_dd'], format='%d/%m/%Y', errors='coerce')
df['stm_fh_ddt'] = pd.to_datetime(df['stm_fh_ddt'], format='%d/%m/%Y %H:%M:%S', errors='coerce')

# Combine date and time columns to a datetime column
df['stm_aanntpl_tijd'] = df['stm_aanntpl_tijd'].astype('str')
df['stm_aanntpl_dd'] = df['stm_aanntpl_dd'].astype('str')
df["aanntpl_ddt"] = df["stm_aanntpl_dd"] + " " + df["stm_aanntpl_tijd"].apply(lambda x: x.split(' ')[-1])
df['aanntpl_ddt'] = pd.to_datetime(df['aanntpl_ddt'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
df = df.dropna(subset=['aanntpl_ddt'])

In [ ]:
# Maak een kolom met de duur van de aannemer ter plaatse tot functieherstel
df['anm_tot_fh'] = df['stm_fh_ddt'] - df['aanntpl_ddt']
df['anm_tot_fh'] = df['anm_tot_fh'].apply(lambda x: x.seconds/60 + x.days * (24*60))
df = df.dropna(subset=['anm_tot_fh'])

We halen negatieve waardes voor de targetvariabele uit de dataset. \
Deze zijn voor ons model, wat de functiehersteltijd wilt berekenen als de aannemer
ter plaatse is gekomen niet relevant. Het probleem is dan namelijk al opgelost. \
\
Verder is ons verteld dat wij storingen met een verwachte functieherstelduur van korter
dan 5 minuten of langer dan 8 uur mogen weglaten. \ 
Deze storingen zijn óf zodanig snel opgelost dat deze niet nuttig zijn om een voorspelling voor te doen, \
óf zodanig lang, dat het treinverkeer toch niet snel zal rijden, en het beter is om af te wachten.

In [ ]:
# We halen negatieve waardes voor de targetvariabele uit de data
df = df[df['anm_tot_fh'] >= 0]
# We halen prognoses voor korter dan 5 minuten en langer dan 8 uur uit de data
df = df[(df['anm_tot_fh'] >= 5) & (df['anm_tot_fh'] <= 480)]

In [ ]:
def remove_outlier(df_in, col_name, k=3):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3 - q1 
    fence_low  = q1 - k * iqr
    fence_high = q3 + k * iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

## Data exploration

### stm_progfh_in_duur

In [ ]:
# Remove nonsense values
dfprog = df
dfprog['stm_progfh_in_duur'] = df['stm_progfh_in_duur'].str.extract('(\d+)', expand=False)
dfprog['stm_progfh_in_duur'] = dfprog['stm_progfh_in_duur'].astype('int32')
dfprog = dfprog[dfprog.stm_progfh_in_duur != 99999999]
dfprog = dfprog[dfprog.stm_progfh_in_duur != 0]

In [ ]:
dfprog = remove_outlier(dfprog, 'stm_progfh_in_duur')

In [ ]:
plt.scatter(dfprog['stm_progfh_in_duur'], dfprog['anm_tot_fh'], s=0.3, alpha=0.1)
plt.title('Initiële prognose aannemer, functieherstelduur')
plt.xlabel('stm_progfh_in_duur')
plt.ylabel('anm_tot_fh')
# plt.plot(np.arange(0,500), np.repeat(60, 500))
# plt.plot(np.arange(0, 500), np.arange(0, 500), color='red')
plt.show()

In [ ]:
df.corr(numeric_only=True)['anm_tot_fh']

In [ ]:
# pd.options.display.max_rows = 1000
with pd.option_context("display.max_rows", 100):
    print(df['stm_oorz_code'].value_counts())

### stm_oorz_groep

In [ ]:
df['stm_oorz_groep'].unique()

In [ ]:
boxplot_data = [df[df.stm_oorz_groep == value]['anm_tot_fh'] for value in df['stm_oorz_groep'].unique()]

fig, ax = plt.subplots()
ax.boxplot(boxplot_data)
ax.set_xticklabels(df['stm_oorz_groep'].unique())
ax.set_ylabel('anm_tot_fh')
plt.show()

In [ ]:
for value in ['ONR-RIB', 'ONR-DERD', 'TECHONV', 'WEER']:
    print(df[df.stm_oorz_groep == value]['anm_tot_fh'].quantile([0.25, 0.5, 0.75]))

### stm_geo_mld

In [ ]:
median_time = []
for value in df['stm_geo_mld'].unique():
    median_time.append(df[(df.stm_geo_mld == value) & (df.stm_oorz_code == 225)]['anm_tot_fh'].median())

median_time = [x for x in median_time if str(x) != 'nan']

plt.boxplot(median_time)
plt.title('anm_tot_fh per geocode voor oorzaakcode 225')
plt.ylabel('Mediaan anm_tot_fh per geocode')

### stm_contractgeb_mld

In [ ]:
contract_time = []
for cont in df['stm_contractgeb_mld'].unique():
    contract_time.append(df[(df.stm_contractgeb_mld == cont) & (df.stm_oorz_code == 225)]['anm_tot_fh'].median())

contract_time = [x for x in contract_time if str(x) != 'nan']

plt.boxplot(contract_time)
plt.title('anm_tot_fh per contractgebied voor oorzaakcode 225')
plt.ylabel('Mediaan anm_tot_fh per contractgebied')
plt.show()
print('')

### stm_prioriteit

In [ ]:
boxplot_data = [df[df.stm_prioriteit == value]['anm_tot_fh'] for value in [1,2,4,5,8,9]]

fig, ax = plt.subplots()
ax.boxplot(boxplot_data)
ax.set_xticklabels([1,2,4,5,8,9])
ax.set_xlabel('Priority label')
ax.set_ylabel('anm_tot_fh')
plt.title('Boxplots for different priority values')
plt.show()

### stm_techn_mld

In [ ]:
boxplot_data = [df[df.stm_techn_mld == value]['anm_tot_fh'] for value in df['stm_techn_mld'].unique()]

fig, ax = plt.subplots()
ax.boxplot(boxplot_data)
ax.set_xticklabels(df['stm_techn_mld'].unique())
ax.set_xlabel('Techniekveld label')
ax.set_ylabel('anm_tot_fh')
plt.title('Boxplots for different techniekveld labels')
plt.show()

## DataPrep klad

### geo_score column

In [ ]:
# For how many different geocodes we perform this operation
len(df['stm_geo_mld'].unique()), len(median_time)

In [ ]:
# geocode to score dictionary
geo_dict = {}
for geocode in df['stm_geo_mld'].unique():
    geo_dict[geocode] = (df[(df.stm_geo_mld == geocode) & (df.stm_oorz_code == 225)]['anm_tot_fh'].median())/max(median_time)

In [ ]:
df['geo_score'] = df['stm_geo_mld'].apply(lambda x: geo_dict[x])

### contractgeb_score

In [ ]:
len(df['stm_contractgeb_mld'].unique()), len(contract_time)

In [ ]:
# contractgebied to score dictionary
contrgb_dict = {}
for contr_geb in df['stm_contractgeb_mld'].unique():
    contrgb_dict[contr_geb] = (df[(df.stm_contractgeb_mld == contr_geb) & (df.stm_oorz_code == 225)]['anm_tot_fh'].median())/max(contract_time)

In [ ]:
df['contractgeb_score'] = df['stm_contractgeb_mld'].apply(lambda x: contrgb_dict[x] if str(x) != 'nan' else np.nan)

In [ ]:
df[['stm_contractgeb_mld', 'contractgeb_score']].sample(20)

In [ ]:
df['stm_contractgeb_mld'].isna().sum()

### stm_oorz_groep

In [ ]:
df['stm_oorz_groep'].isna().sum()

In [ ]:
orz_dict = {'TECHONV':0, '':0, 'ONR-RIB':0, 'ONR-DERD':1, 'WEER':0}
df['oorz_groep_cluster'] = df['stm_oorz_groep'].apply(lambda x: orz_dict[x])

### stm_prioriteit

Willen we dit doen?

### stm_techn_mld

In [ ]:
techncl_dict = {'':1, 'S':1,'B':1,'K':1,'E':1,'T':1,'X':1,'I':1, 
                'P':0,'O':0,'G':0,
                'M':2,'A':2, 
                'H':3}
df['techn_cluster'] = df['stm_techn_mld'].apply(lambda x: techncl_dict[x])